In [1]:
%matplotlib ipympl
def figure(name, nrows=1, ncols=1, *args, **kwargs):
    plt.close(name)
    return plt.subplots(nrows, ncols, num=name, *args, **kwargs)

import numpy as np
import pylab as plt
plt.style.use('default')

In [2]:
from scipy.integrate import solve_ivp
from ipywidgets import HBox, IntSlider, FloatSlider
from matplotlib import patches

In [3]:
def create_rectangle(ax, coords,  **kwargs):
  # coords are given in the xlim, ylim style, i.e., the start and stop value rather then start and width/height
  default = {'linewidth':1.5,
             'edgecolor':'k',
             'alpha':.9,
             'facecolor':'none',
             'zorder':100}

  # overwrite default values if defined
  kwargs = {**default, **kwargs}
  
  rect = patches.Rectangle((coords[0][0], coords[1][0]), 
           coords[0][1] - coords[0][0], coords[1][1] - coords[1][0],
           **kwargs)
  ax.add_patch(rect)
  return rect

# Rotation auf Tischplatte

In [4]:
l = 5
M = 1
L = 1
g = 9.81
m = 1

In [5]:
def get_slider():
    Ls = FloatSlider(
           orientation='horizontal', description='L: ',
           value=1, min=0, max=10,
    )
    Ms = FloatSlider(
           orientation='horizontal', description='M: ',
           value=1, min=0, max=10,
    )
    L = Ls.value
    M = Ms.value
    r = np.linspace(1e-10, 2, 1000)
    
    fig, ax = figure('potential', 1, 1, figsize=(10, 5))
    lines = [ax.plot(r, M*g*r)[0],
             ax.plot(r, L**2 / (2*m*r**2))[0],
             ax.plot(r, L**2 / (2*m*r**2) + M*g*r, 'k')[0],
            ]  

    ax.set_xlim([-0.1, 2.1])
    ax.set_ylim([-1, 20])
    rect = create_rectangle(ax, [[0, 1.5], [-2, 21]],
                           facecolor='#1e1e1e', alpha=0.1)
    
    def redraw(change):
        L = Ls.value
        M = Ms.value
        lines[0].set_data(r, M*g*r)
        lines[1].set_data(r, L**2 / (2*m*r**2))
        lines[2].set_data(r, L**2 / (2*m*r**2) + M*g*r)
        fig.canvas.draw()
        fig.canvas.flush_events()
    
    Ls.observe(redraw, names='value')
    Ms.observe(redraw, names='value')
    return HBox([Ms, Ls])

get_slider()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Gleiten im Kegel

In [6]:
from scipy.integrate import odeint
from numpy import cos, sin 

In [7]:
def derivative(y, t):
    dy = np.zeros(4)
    dy[0] = y[1]
    dy[1] = y[0] * (sin(al)*y[3])**2 - g*cos(al)
    dy[2] = y[3]
    dy[3] = - 2 * y[1]/y[0] * y[3] 
    
#     v2 = y[1]**2 + (y[0]*sin(al)*dy[2])**2
    dy[1] -= k * (y[1])**2
    dy[3] -= k * (y[0]*y[3]*sin(al))**2
    
    return dy

In [8]:
L0 = 1
m = 1
g = 9.81
al = np.pi/6
h0 = (L0**2 / (m**2 * cos(al)*sin(al)**2 * g))**(1./3)
dphi0 = L0 / (m * h0**2*sin(al)**2)
k = 0

Y0 = [h0, 0, 0, dphi0]
t = np.linspace(0, 10, 1000)

In [9]:
sol1 = odeint(derivative, Y0, t)
h0 = 1.05 * h0
dphi0 = L0 / (m * h0**2*sin(al)**2)
Y0 = [h0, 0, 0, dphi0]
sol2 = odeint(derivative, Y0, t)

In [10]:
fig, ax = figure('test_impulse')
ax.plot(t, sol1[:, 0])
ax.plot(t, sol2[:, 0])
ax.set_xlabel('t in s')
ax.set_ylabel('h in m')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'h in m')

## Reibung

In [11]:
k = .4
Y0 = [h0, 0, 0, dphi0]
t = np.linspace(0, 100, 1000000)

In [12]:
sol3 = odeint(derivative, Y0, t)

fig, ax = figure('friction', 1, 2, figsize=(12, 4))
ax[0].plot(t, sol3[:, 0])
ax[0].set_xlabel('t in s')
ax[0].set_ylabel('h in m')
ax[1].plot(t, sol3[:, 3])
ax[1].set_xlabel('t in s')
ax[1].set_ylabel('$\dot{\phi}$ in rad/s')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, '$\\dot{\\phi}$ in rad/s')

In [13]:
from mpl_toolkits.mplot3d import Axes3D
import mpl_toolkits.mplot3d.axes3d as p3
import matplotlib.animation as ani

In [14]:
def get_cart_coords(y):
    x = np.zeros((3, y.shape[1]))
    x[0] = y[0]*sin(al) * cos(y[2])
    x[1] = y[0]*sin(al) * sin(y[2])
    x[2] = y[0]*cos(al)
    return x

In [15]:
coords = get_cart_coords(sol3.T)

plt.close('test_3d_traj')
fig = plt.figure('test_3d_traj')
ax = fig.add_subplot(111, projection='3d')

ax.plot(coords[0], coords[1], coords[2])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Animation

In [16]:
from IPython.display import HTML, Audio

In [17]:
HTML("""
<div align="middle">
<video width="80%" controls>
      <source src="test.mp4" type="video/mp4">
</video></div>""")

### Sound

In [18]:
from scipy.io.wavfile import write

In [27]:
data = (sin(sol3[:, 2]*100)  +
        sin(sol3[:, 2]*100*4/3) + 
        sin(sol3[:, 2]*100*5/3) +
        0.2 * (np.random.random(1000000)-1)
       ) * sol3[:, 1]
scaled = np.int16(data/np.max(np.abs(data)) * 32767)
write('test.wav', 10000, scaled)

In [28]:
Audio("test.wav")

In [21]:
np.random.random(100)

array([0.19143055, 0.75784961, 0.80392828, 0.07157059, 0.60342776,
       0.27022085, 0.59944329, 0.18246094, 0.35716004, 0.0091867 ,
       0.57058939, 0.16013781, 0.31289693, 0.5902403 , 0.43243432,
       0.96115404, 0.97856742, 0.05550034, 0.08460021, 0.43014875,
       0.37473704, 0.70154393, 0.13800278, 0.73276093, 0.07384373,
       0.42412825, 0.22551851, 0.14964924, 0.3865839 , 0.59548397,
       0.99561482, 0.83528113, 0.83270819, 0.26761792, 0.56674843,
       0.34204903, 0.29994225, 0.73535336, 0.16969553, 0.42642651,
       0.39109455, 0.6784798 , 0.87366153, 0.18377252, 0.16221153,
       0.82427381, 0.77499941, 0.5833914 , 0.61821737, 0.15300495,
       0.37440456, 0.55984437, 0.24369626, 0.71198934, 0.74840554,
       0.97073464, 0.58694825, 0.4749348 , 0.90745822, 0.81901234,
       0.71281059, 0.26813123, 0.42471167, 0.55311123, 0.66876941,
       0.53853582, 0.47353588, 0.06261856, 0.28559362, 0.66304932,
       0.03254288, 0.51701492, 0.96397536, 0.69444086, 0.63002